In [1]:
import cv2
import PIL 
from facenet_pytorch import MTCNN
from PIL import Image,ImageDraw

import torch
import numpy as np
from torchvision import datasets, models, transforms

In [2]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize((224,224)),
                                 transforms.ToTensor()]),
    'test': transforms.Compose([transforms.Resize((224,224)),
                                 transforms.ToTensor()]),
}

In [3]:
# check if gpu is available
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Load Model
model=torch.load('face_mask_model.pth')
model.eval()

print(device)

cuda


In [4]:
mtcnn=MTCNN(keep_all=True, device=device)

In [12]:
cap=cv2.VideoCapture(0)

while True:
    ret,image=cap.read()
    image = cv2.flip(image, 1)
    
    frame=image.copy()
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    boxes,_=mtcnn.detect(frame)
    
    color=(0,255,0)
    
    if boxes is not None:
        for box in boxes:
            startX,endX=int(box[0])-1,int(box[2])+1
            startY,endY=int(box[1])-1,int(box[3])+1
            cv2.rectangle(image,(box[0],box[1]),(box[2],box[3]) , color, 2)
            img = PIL.Image.fromarray(frame)
            with torch.no_grad():
                img = data_transforms['test'](img)
                img = img.to(device)
                img = img.view(-1, 3, 224, 224)
                res = model(img)
            txt = 'Mask' if torch.argmax(res) == 0 else 'No Mask'
            color = (0, 255, 0)
            cv2.putText(image, txt, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
    
    cv2.imshow("frame",image)
    k=cv2.waitKey(1) & 0xFF
    if k == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()